In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# numerical/grid parameters
Nx = 128
Ny = 128
dx = 1.0
dy = 1.0
dt = 0.5
nSteps = 2000
printSteps = 50

In [ ]:
# physical parameters
# f = a c^2 (1-c)^2
# df/dc = 2 a c (1-c)^2 - 2 a c^2 (1-c) = 2 a c (1-c) (1-2c)
# 

# physics based parametrs
A = 1.0
kappa = 1.0
M = 1.0
c0 = 0.5
noiseAmp = 0.005
c = c0 * np.ones((Nx, Ny))

np.random.seed(714)
# add random noise with mean 0 and std 0.1
noise = 2 * np.random.randn(Nx, Ny) - 1.0
noise = noise - np.mean(noise)
noise *= noiseAmp

c = c + noise
print(c.mean())

In [ ]:
plt.imshow(noise)
plt.colorbar()

In [ ]:
# fourier transform
# kx = np.fft.fftfreq(Nx, dx)
kx = np.fft.fftshift(np.fft.fftfreq(Nx, dx))
kx = 2 * np.pi * kx
ky = np.fft.fftshift(np.fft.fftfreq(Ny, dy))
ky = 2 * np.pi * ky
kx2 = kx*kx
ky2 = ky*ky
k2 = kx2 + ky2
k4 = k2*k2

# solve the PDE
c_hat = np.fft.fft2(c)
c_hat = np.fft.fftshift(c_hat)

g = 2 * A * c * (1 - c) * (1 - 2 * c)
g_hat = np.fft.fft2(g)
g_hat = np.fft.fftshift(g_hat)

c_hat_new = (c_hat - M * dt * k2 * g_hat) / (1 + 2 * M * kappa * k4 * dt)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define spatial resolution and grid sizes
Nx, Ny = 256, 256  # Grid sizes in x and y directions
dx, dy = 1.0, 1.0  # Grid spacing in x and y directions
dt = 0.01  # Time step
M = 1.0  # Diffusion coefficient
A = 1.0  # Parameter for the reaction term
kappa = 0.1  # Diffusion parameter for the higher-order term
timesteps = 100  # Number of timesteps to run the simulation

# Create the grid in Fourier space
kx = np.fft.fftfreq(Nx, dx)  # Frequency in x direction
kx = np.fft.fftshift(kx)  # Shift to center the zero frequency
kx = 2 * np.pi * kx  # Convert to wave numbers

ky = np.fft.fftfreq(Ny, dy)  # Frequency in y direction
ky = np.fft.fftshift(ky)  # Shift to center the zero frequency
ky = 2 * np.pi * ky  # Convert to wave numbers

# Compute k^2 and k^4
kx2 = kx**2
ky2 = ky**2
k2 = kx2 + ky2  # k^2
k4 = k2 * k2  # k^4

# Initial condition for c (you can choose your own initial condition)
c = np.random.rand(Nx, Ny)  # This is just an example; replace with actual initial condition

# Fourier transform of c
c_hat = np.fft.fft2(c)  # 2D Fourier transform of c
c_hat = np.fft.fftshift(c_hat)  # Shift zero frequency to center

# Time-stepping loop
for t in range(timesteps):
    # Reaction term g = 2 * A * c * (1 - c) * (1 - 2 * c)
    g = 2 * A * c * (1 - c) * (1 - 2 * c)

    # Fourier transform of g
    g_hat = np.fft.fft2(g)  # 2D Fourier transform of g
    g_hat = np.fft.fftshift(g_hat)  # Shift zero frequency to center

    # Update rule for c_hat using implicit time-stepping
    c_hat_new = (c_hat - M * dt * k2 * g_hat) / (1 + 2 * M * kappa * k4 * dt)

    # Optional: Inverse FFT to get the updated field in real space
    c_new = np.fft.ifft2(np.fft.ifftshift(c_hat_new))  # Inverse FFT to get c in real space
    c_new = np.abs(c_new)  # Take absolute value if necessary, for real fields

    # Update the field for the next iteration
    c = c_new.copy()  # Copy the updated field back to c

    # Optional: Display the solution at certain timesteps (for visualization)
    if t % 10 == 0:  # Display every 10 timesteps
        plt.imshow(c, cmap='viridis', origin='lower')
        plt.title(f"Timestep {t}")
        plt.colorbar()
        plt.show()

# After the loop, `c` holds the final solution
